In [ ]:
import numpy as np
import cv2
import pickle
import os
from shutil import rmtree

In [ ]:
# Load model Haar-cascade Detection
face_cascade = cv2.CascadeClassifier("D:\Master\Thesis\src\opencv-master\data\haarcascades\haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier('D:\Master\Thesis\src\opencv-master\data\haarcascades\haarcascade_eye.xml')

# video_path = './../../data/video/ch3_26_3_1030_1130.mp4'

# # load video
# cap = cv2.VideoCapture(video_path)

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
fgbg = cv2.createBackgroundSubtractorMOG2()

In [ ]:
#Calcualte Background

def getForeground(alpha, frame):
    # Set background as first frame
    backGroundModel = frame
    # Update back ground model, apply the background averaging formula:
    # NEW_BACKGROUND = CURRENT_FRAME * ALPHA + OLD_BACKGROUND * (1 - APLHA)
    backGroundModel = frame * alpha + backGroundModel * (1- alpha)
    
    return cv2.absdiff(backGroundModel.astype(np.uint8), frame)

In [ ]:
gray_pre = np.zeros((1080,1920),dtype=np.uint8)
pre_face = 0

frame_pos_list = []

VIDEO_PATH = './../../data/video'
RAW_IMG = './../../data/result/raw_img'
DETECTED_IMG = './../../data/result/detected_img'
FRAME_LIST = './../../data/result/frame_list'

for root, dirs, files in os.walk(VIDEO_PATH):
    files.sort()
    
    for idx, name in enumerate(files):
        video_path = os.path.join(root, name)
        video_name = name.split('.')[0]
        
        # Reset output dir
        RAW_IMG_DIR = os.path.join(RAW_IMG, video_name)
        if os.path.exists(RAW_IMG_DIR):
            rmtree(RAW_IMG_DIR)
        os.mkdir(RAW_IMG_DIR)
        
        DETECTED_IMG_DIR = os.path.join(DETECTED_IMG, video_name)
        if os.path.exists(DETECTED_IMG_DIR):
            rmtree(DETECTED_IMG_DIR)
        os.mkdir(DETECTED_IMG_DIR)

#         FRAME_LIST_DIR = os.path.join(FRAME_LIST, video_name)
#         if os.path.exists(FRAME_LIST_DIR):
#             rmtree(FRAME_LIST_DIR)
#         os.mkdir(FRAME_LIST_DIR)
        
        # load video
        cap = cv2.VideoCapture(video_path)

        while(cap.isOpened()):
            ret, frame = cap.read()
            print ('frame nunber: %d' % cap.get(1))

            if ret == True:

                """
                # Method 1: Use createBackgroundSubtractorMOG2() function to calculate background.
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                fgmask = fgbg.apply(gray, learningRate=0.01)
                fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

                ret, mask = cv2.threshold(fgmask, 128, 255, cv2.THRESH_BINARY)

                dif_rate = np.sum(mask)/(1080 * 1920)

                if((dif_rate < 0.04) & (pre_face == 0)):
                    continue

                pre_face = 0
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                """

                #"""
                # Method 2: Use subtract function in opencv
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                dif = cv2.subtract(gray, gray_pre)
                th, dst = cv2.threshold(dif, 2, 1, cv2.THRESH_BINARY)
                dif_rate = np.sum(dst) / (1080 * 1920)
                gray_pre = gray

                if((dif_rate < 0.04) & (pre_face == 0)):
                    continue

                pre_face = 0
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                #"""

                """
                # Method 3: Use absDiff to calculate background model.
                ###Get the foreground
                foreGround = getForeground(0.01, frame)
                foreGround = cv2.cvtColor(foreGround, cv2.COLOR_BGR2GRAY)

                #Apply thresholding on the background and display the resulting mask
                ret, mask = cv2.threshold(foreGround, 128, 255, cv2.THRESH_BINARY)

                dif_rate = np.sum(mask)/(1080 * 1920)

                if((dif_rate < 0.04) & (pre_face == 0)):
                    continue

                pre_face = 0
                faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                """

                if(len(faces) > 0):
                    pre_face = 1

                    frame_idx = cap.get(1)
                    frame_pos_list.append(frame_idx)
                    
                    name_raw = os.path.join(RAW_IMG_DIR, 'frame%d' % frame_idx + '.png')
                    #name_raw = './../../data/raw_img/'+ 'name' + 'frame%d' % frame_idx + '.png'
                    
                    cv2.imwrite(name_raw, frame)

                    for (x, y, w, h) in faces:
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                        roi_gray = gray[y:y+h, x:x+w]
                        roi_color = frame[y:y+h, x:x+w]

                        eyes = eye_cascade.detectMultiScale(roi_gray)
                        for (ex, ey, ew, eh) in eyes:
                           cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

                    # Save the detected image
                    name_detected = os.path.join(DETECTED_IMG_DIR, 'frame%d' % frame_idx + '.png')
                    cv2.imwrite(name_detected, frame)
            else:
                break

        cap.release()
        
        # Save the frame list have face detect
        frame_list_path = os.path.join(FRAME_LIST, video_name +  '.pkl')
        with open(frame_list_path, 'wb') as f:
            pickle.dump(frame_pos_list, f)
 